# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.



## Откройте и изучите файл


In [1]:
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Исправим тип данных для колонки messages на int, так как там должно быть целое число:

In [6]:
df['messages']=df['messages'].astype('int')

Посмотрим на соотношение тарифов, чтобы в дальнейшем можно было сопоставить с предсказанием тестовой модели:

In [7]:
df.is_ultra.value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Также просмотрим пропуски и дубликаты:

In [8]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [9]:
df.duplicated().sum()

0

**Вывод:**

Пропуски и дубликаты отсутствуют, типы данных соответствуют колонкам. Наша тестовая модель не должна предсказывать с точностью 0.3 или 0.69. Можно приступить к исследованию.

## Разбейте данные на выборки

Создадим переменные для признаков и извлечем целевой признак.

Также разобьем данные в соотношении 60:20:20.

In [10]:
df_train, df_ch = train_test_split(df, test_size=0.40, random_state=12345)

In [11]:
df_test, df_valid  = train_test_split(df_ch, test_size=0.50, random_state=12345)

In [12]:
print('Размер обучающей выборки составляет:', df_train.shape[0])
print('Размер тестовой выборки составляет:', df_test.shape[0])
print('Размер валидационной выборки составляет:', df_valid.shape[0] )

Размер обучающей выборки составляет: 1928
Размер тестовой выборки составляет: 643
Размер валидационной выборки составляет: 643


Создадим переменные для признаков для каждой из выборок:
    

In [13]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

**Вывод:**

1. Данные разбили на обучающую, тестовую и валидационную выборки в соотношении 60:20:20
2. Целевой признак является категориальным (только два тарифа), то есть  в дальнейшим необходимо решить задачу классификации;
3. Обучение будет производиться на df_train, валидация - на df_valid, для оценки качества модели - на df_test.

## Исследуйте модели

Начнем исследование с **"дерева решений":**

In [14]:
best_depth = 0
best_accuracy = 0
best_model = None


for depth in range(1, 100):
    model = DecisionTreeClassifier(random_state=23042020, max_depth=depth)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
   
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy

print('Лучшая глубина дерева:', best_depth,'   Лучшее качество:', best_accuracy)  

Лучшая глубина дерева: 7    Лучшее качество: 0.7962674961119751


Также можно рассмотреть другие гиперпараметры, оставим лучшую глубину 7. Заменим гиперпараметр gini на entropy в  criterion, а  splitter вместо гиперпараметра best поставим  random: 

In [15]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7,criterion='entropy')
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Качество',accuracy)

Качество 0.7962674961119751


In [16]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7, splitter='random')
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Качество',accuracy)

Качество 0.776049766718507


In [17]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7,criterion='entropy', splitter='random')
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Качество',accuracy)

Качество 0.7900466562986003


Гиперпараметр random показывает себя хуже, чем best; gini вместе с best - лучше, чем entropy и random.

**"Случайный лес":**


In [18]:
best_est = 0
best_accuracy = 0
best_model = None

for est in range(1, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators= est)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)

    if accuracy > best_accuracy:
        best_est = est
        best_accuracy = accuracy

print('Лучшее количество деревьев:', best_est,'   Лучшее качество:', best_accuracy) 

Лучшее количество деревьев: 48    Лучшее качество: 0.7962674961119751


Также посмотрим "Случайный лес" с такими гиперпараметрами, как bootstrap=False и warm_start=True:

In [19]:
model = RandomForestClassifier(random_state=12345, n_estimators=48, bootstrap=False)
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Точность',accuracy)

Точность 0.7713841368584758


In [20]:
model = RandomForestClassifier(random_state=12345, n_estimators=48,  warm_start=True)
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Точность',accuracy)

Точность 0.7962674961119751


In [21]:
model = RandomForestClassifier(random_state=12345, n_estimators=48, bootstrap=False, warm_start=True)
model.fit(features_train, target_train)
valid_predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, valid_predictions)
print('Точность',accuracy)

Точность 0.7713841368584758


**"Логическая регрессия":**

In [22]:
import warnings
warnings.filterwarnings('ignore')

best_max_iteration = 0
best_accuracy = 0
best_model = None

for max_it in range(1,100):
    model = LogisticRegression(random_state=23042020, max_iter=max_it)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
    
    if accuracy > best_accuracy:
        best_max_iteration = max_it
        best_accuracy = accuracy

print('Лучшее количество итерраций:', best_max_iteration,'   Лучшее качество:', best_accuracy) 

Лучшее количество итерраций: 21    Лучшее качество: 0.6982892690513219


**Вывод**:
1. "Дерево решений" (при лучшей глубине 7 и gini) и "Случайный лес" (при количестве деревьев 48) показывают лучший результат - по 79,6% правильных ответов;
2. у модели "Логическая регрессия" - 69,8%;
3. Изменение в гиперпараметрах для "дерева решений" не привело к увеличению правильных ответов.

NB! Поскольку скорость обработки у "дерева решений" выше, чем у "случайного леса" при одинаковых показателях.

## Проверьте модель на тестовой выборке

Поскольку показатели точности для моделей "дерева решенией" и "случайного леса" на валидационной выборке оказались идентичными, то проверку именно их модели будем проверять на тестовой выборке:

In [23]:
tree_model = DecisionTreeClassifier(random_state=12345, max_depth=7,criterion='gini')
tree_model.fit(features_train, target_train)
test_predictions = tree_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Качество дерева решений на тестовой выборке: ',accuracy)

Качество дерева решений на тестовой выборке:  0.7822706065318819


In [24]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=48)
forest_model.fit(features_train, target_train)
test_predictions = forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Качество случайного леса на тестовой выборке: ',accuracy)

Качество случайного леса на тестовой выборке:  0.7916018662519441


**Вывод:**
1. Точность лучшей тестовой модели - тестовая случайного леса - составила 79.2%;
2. Также можно предположить, что тестовые модели (из-за схожести с моделями валидации) являются слабо переобученными.

## Общий вывод:
1. Были изучены и скорректированы данные;
2. Сделана разбивка на три выборки: обучающая, валидация и тестовая;
3. Были рассмотрены три модели классификации (дерево решений, случайный лес и логическая регрессия), изменены гиперпараметры;
4. Было выявлено, что наилучшей валидационной и тестовой моделью является случайный лес (79,2%), что выше, чем распределение тарифов в данных, то есть тестовая модель не предсказывает 0 или 1.



<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:


Владислав, здорово что в конце расписаны все этапы работы. Это важно потому что когда проект захочет посмотреть будущий работодатель (или начальник), у него может не быть времени на подробный разбор кода. Вероятнее всего он бегло просмотрит код, а из общего вывода захочет получить представление о всей работе.



</div>



<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>




Владислав, у тебя старательно выполненная работа, все четко, осмысленно. Вижу у тебя получается в ML,  дальше будет интересней )  



Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить): 





- использовать stratify
- добавить график метрики    
- посоветовал посмотреть в сторону GridSearchCV
- как можно улучшить результаты 




Замечания на будущее:
    

- Комментарии к коду отсутствуют - коллеги могут не понять хода твоих мыслей, пока вроде код простой, но стоит держать в голове что дальше код усложнится.  

    

Посылаю на вторую итерацию, чтобы посмотреть твое решение бонусного задания (если захочешь доделать) и ответить на вопросы если они появятся




</div>